In [14]:
import numpy as np
import pandas as pd
import pickle

import geopandas as gpd
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from os import path
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely import wkt
from tqdm import tqdm

In [15]:
geolocator = Nominatim(user_agent='housing-qc')

In [16]:
bounding_territories_df = pd.read_csv('../data/references/handmade/bounding-territories.csv')
bounding_territories_df.sample(5)

,Bounding Territory,Display Name,Bounding Type,Bounding Population,GeoPy Index
72,Pierrefonds-Roxboro,Pierrefonds-Roxboro,Montréal Borough,70000,0
77,Saint-Léonard,Saint-Léonard,Montréal Borough,80000,0
9,Saint-Augustin-de-Desmaures,Saint-Augustin-de-Desmaures,Municipality,20000,0
27,"[Arthabaska,L'Érable Centre-du-Québec]",Arthabaska & L'Érable,Regional County Municipality (RCM),95000,-1
11,"[La Côte-de-Beaupré,L'Île-d'Orléans]",La Côte-de-Beaupré & L'Île-d'Orléans,Regional County Municipality (RCM),35000,0


In [17]:
def get_bounding_polygons(geolocator: Nominatim, bounding: str, geopy_index: int):
    polygons = []
    substracts = []
    for location in bounding.replace("[", "").replace("]", "").split(","):
        if location.startswith("-"):
            geocodes = geolocator.geocode(location[1:] + ', QC', geometry='wkt', exactly_one=False)
            for geocode in geocodes:
                substracts.append(wkt.loads(geocode.raw['geotext']))
        else:
            geocode = geolocator.geocode(location + ', QC', geometry='wkt', exactly_one=False)[geopy_index]
            polygons.append(wkt.loads(geocode.raw['geotext']))

    return polygons, substracts

In [18]:
def get_bounding_polygon(geolocator: Nominatim, bounding: str, geopy_index: int):
    polygons, substracts = get_bounding_polygons(geolocator, bounding, geopy_index)
    return gpd.GeoSeries(unary_union(polygons).difference(unary_union(substracts))).simplify(tolerance=0.001).iloc[0]

In [19]:
def get_polygons(geolocator: Nominatim, bounding_territories_df: pd.DataFrame):
    polygons = []
    for _, location in tqdm(bounding_territories_df.iterrows(), desc="Building GeoSeries", total=bounding_territories_df.shape[0]):
        polygons.append(get_bounding_polygon(geolocator, location["Bounding Territory"], location["GeoPy Index"]))
    return polygons

In [20]:
output = '../data/processed/locations/location-polygons.gpkg'

if path.exists(output):
    unknown_locations = []
    polygons_gdf = gpd.read_file(output)
else:
    polygons = get_polygons(geolocator, bounding_territories_df)
    d = {'location': bounding_territories_df["Display Name"].to_list(), 'geometry': polygons}
    polygons_gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")
    polygons_gdf.to_file(output)

In [21]:
polygons_gdf.sample(10)

,location,geometry
83,Trois-Rivières,"POLYGON ((-72.77942 46.31866, -72.77905 46.316..."
37,Pierre-de-Saurel,"POLYGON ((-73.20407 45.87555, -73.12387 45.817..."
30,Brossard,"POLYGON ((-73.52372 45.47791, -73.51924 45.468..."
87,L'Assomption,"POLYGON ((-73.58056 45.82039, -73.55007 45.816..."
18,Les Appalaches,"POLYGON ((-71.68612 45.96671, -71.56214 45.906..."
35,La Haute-Yamaska,"POLYGON ((-72.87613 45.28444, -72.86160 45.286..."
93,Hull,"POLYGON ((-75.79685 45.48246, -75.79399 45.478..."
76,Saint-Laurent,"POLYGON ((-73.77388 45.50051, -73.77185 45.499..."
66,Le Sud-Ouest,"POLYGON ((-73.63515 45.45074, -73.62886 45.448..."
102,Mirabel,"MULTIPOLYGON (((-74.20253 45.49178, -74.25107 ..."
